In [ ]:
#Algorítimo de Árvores de decisão

In [2]:
# Importação das bibliotecas
import pandas as pd
import requests
import numpy as np
import re
import sqlalchemy
import mysql.connector

In [3]:
conn = mysql.connector.connect(host="", user="", passwd="", db="zabbix")
print(conn)

In [4]:
cursor = conn.cursor(dictionary=True)
cursor.execute("SELECT DISTINCT p.eventid,p.name as problema,h.name,e.acknowledged,p.severity FROM problem p,functions f,items i,hosts_groups hg,hosts h,events e WHERE p.source='0' AND p.object='0' AND p.objectid=f.triggerid AND f.itemid=i.itemid AND i.hostid=hg.hostid AND i.hostid=h.hostid AND i.flags IN (0,4) AND i.status IN (0,1) AND p.eventid=e.eventid AND p.r_eventid IS NULL ORDER BY p.eventid;")
result = cursor.fetchall()

for item in result:
    item['problema'] = item['problema'].decode()
    item['name'] = item['name'].decode()


In [1]:
zabbix = pd.DataFrame(result)
zabbix

NameError: name 'pd' is not defined

In [6]:
# Conversão dos atributos categóricos para atributos numéricos
for col in zabbix.columns:
    if zabbix[col].dtype == 'object':
        zabbix[col] = zabbix[col].astype('category').cat.codes

In [7]:
#zabbix.head(10)

In [8]:
dados = zabbix.iloc[:,1:5].to_json(orient='split')

In [9]:
previsao = requests.post(url='http://localhost:1234/invocations', headers={'Content-Type' : 'application/json' , }, data=dados)

In [10]:
previsao

<Response [200]>

In [11]:
zabbix.shape[0]

998

In [12]:
arr = zabbix.to_numpy()
for x in range(0,zabbix.shape[0]-1):
    if arr[x][4] != previsao.json()[x]:
        cursor.execute("UPDATE problem SET severity = %s WHERE eventId = %s", (int(previsao.json()[x]), int(arr[x][0])))
        conn.commit()
print('Update finalizado!')

Update finalizado!


In [13]:
#cursor = conn.cursor(dictionary=True)
cursor.execute("UPDATE problem SET severity = 4 WHERE eventid = 373276504;")
conn.commit()

In [14]:
#cursor = conn.cursor(dictionary=True)
#cursor.execute("SELECT eventid,name,severity from problem where eventid = 373276504;")
#cursor.fetchall()

In [15]:
cursor.close()
conn.close()
print("Done.")

Done.
